## Season 4 Site-specific Growing Degree Day Values
* gdd to seedling emergence
* gdd to flowering
* gdd to flag leaf emergence

In [2]:
import datetime
import numpy as np
import pandas as pd

In [5]:
# %pwd
# %cd '/Users/ejcain/UA-AG/phenotypes/terraref-datasets/'

/Users/ejcain/UA-AG/phenotypes/terraref-datasets


In [6]:
df_0 = pd.read_csv('data/raw/mac_season_4.csv', low_memory=False)
print(df_0.shape)
# df_0.head()

(372363, 39)


,Unnamed: 0,checked,result_type,id,citation_id,site_id,treatment_id,sitename,city,lat,...,n,statname,stat,notes,access_level,cultivar,entity,method_name,view_url,edit_url
0,1,0,traits,6001958927,6.000000e+09,6000005673,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 5,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI181083,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
1,2,0,traits,6001958928,6.000000e+09,6000005676,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 6,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI564163,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
2,3,0,traits,6001958931,6.000000e+09,6000005685,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 9,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI52606,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
3,4,0,traits,6001958933,6.000000e+09,6000005691,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 11,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI533792,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
4,5,0,traits,6001958936,6.000000e+09,6000005700,6.000000e+09,MAC Field Scanner Season 4 Range 11 Column 14,Maricopa,33.074907,...,NaN,NaN,NaN,NaN,2,PI535794,NaN,Visual assessment of leaf dessication,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...


In [11]:
# df_0.columns

In [10]:
cols_to_drop = ['Unnamed: 0', 'checked', 'result_type', 'id', 'citation_id', 'site_id', 'treatment_id', 'city', 
                'scientificname', 'commonname', 'genus', 'species_id', 'cultivar_id', 'author', 'citation_year', 
                'treatment', 'time', 'raw_date', 'month', 'year', 'dateloc', 'trait_description', 'units', 'n',
                'statname', 'stat', 'notes', 'access_level', 'entity', 'method_name', 'view_url', 'edit_url']

In [12]:
df_1 = df_0.drop(labels=cols_to_drop, axis=1)
print(df_1.shape)
# df_1.head()

(372363, 7)


,sitename,lat,lon,date,trait,mean,cultivar
0,MAC Field Scanner Season 4 Range 11 Column 5,33.074907,-111.974982,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI181083
1,MAC Field Scanner Season 4 Range 11 Column 6,33.074907,-111.974966,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI564163
2,MAC Field Scanner Season 4 Range 11 Column 9,33.074907,-111.974917,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI52606
3,MAC Field Scanner Season 4 Range 11 Column 11,33.074907,-111.974884,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI533792
4,MAC Field Scanner Season 4 Range 11 Column 14,33.074907,-111.974835,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI535794


#### Convert dates from string to datetime

In [13]:
new_dates = []

for d in df_1.date.values:
    
    if 'Phoenix' in d:
        new_name = d[:-18]
        new_dates.append(new_name)
    
    else:
        new_name = d
        new_dates.append(new_name)
        
print(df_1.shape[0])
print(len(new_dates))

372363
372363


In [14]:
iso_format_dates = pd.to_datetime(new_dates)

In [15]:
df_2 = df_1.copy()

df_2['date_1'] = iso_format_dates
# df_2.head()

,sitename,lat,lon,date,trait,mean,cultivar,date_1
0,MAC Field Scanner Season 4 Range 11 Column 5,33.074907,-111.974982,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI181083,2017-06-14
1,MAC Field Scanner Season 4 Range 11 Column 6,33.074907,-111.974966,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI564163,2017-06-14
2,MAC Field Scanner Season 4 Range 11 Column 9,33.074907,-111.974917,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI52606,2017-06-14
3,MAC Field Scanner Season 4 Range 11 Column 11,33.074907,-111.974884,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI533792,2017-06-14
4,MAC Field Scanner Season 4 Range 11 Column 14,33.074907,-111.974835,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI535794,2017-06-14


#### Change sitenames to strip ` E` and ` W` subplot designations

In [16]:
sitename_values = df_2.sitename.values
no_e_w_names = []

for name in sitename_values:
    
    if name.endswith(' W') | name.endswith(' E'):
        name = name[:-2]
        no_e_w_names.append(name)
        
    else:
        no_e_w_names.append(name)

In [17]:
df_3 = df_2.copy()
df_3['sitename_1'] = no_e_w_names
# df_3.head()

,sitename,lat,lon,date,trait,mean,cultivar,date_1,sitename_1
0,MAC Field Scanner Season 4 Range 11 Column 5,33.074907,-111.974982,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI181083,2017-06-14,MAC Field Scanner Season 4 Range 11 Column 5
1,MAC Field Scanner Season 4 Range 11 Column 6,33.074907,-111.974966,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI564163,2017-06-14,MAC Field Scanner Season 4 Range 11 Column 6
2,MAC Field Scanner Season 4 Range 11 Column 9,33.074907,-111.974917,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI52606,2017-06-14,MAC Field Scanner Season 4 Range 11 Column 9
3,MAC Field Scanner Season 4 Range 11 Column 11,33.074907,-111.974884,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI533792,2017-06-14,MAC Field Scanner Season 4 Range 11 Column 11
4,MAC Field Scanner Season 4 Range 11 Column 14,33.074907,-111.974835,2017 Jun 14 (America/Phoenix),leaf_desiccation_present,0.0,PI535794,2017-06-14,MAC Field Scanner Season 4 Range 11 Column 14


#### Extra Range and Column Values

In [18]:
df_4 = df_3.copy()

df_4['range'] = df_4['sitename_1'].str.extract("Range (\d+)").astype(int)
df_4['column'] = df_4['sitename_1'].str.extract("Column (\d+)").astype(int)

# df_4.sample(n=7)

,sitename,lat,lon,date,trait,mean,cultivar,date_1,sitename_1,range,column
324842,MAC Field Scanner Season 4 Range 16 Column 14,33.075087,-111.974835,2017 Jun 4,canopy_height,89.000000,PI570254,2017-06-04,MAC Field Scanner Season 4 Range 16 Column 14,16,14
343938,MAC Field Scanner Season 4 Range 31 Column 8,33.075626,-111.974933,2017 Jul 6,leaf_angle_beta,1.786100,PI641821,2017-07-06,MAC Field Scanner Season 4 Range 31 Column 8,31,8
272505,MAC Field Scanner Season 4 Range 26 Column 15,33.075446,-111.974819,2017 Aug 4,qP,0.459000,PI511355,2017-08-04,MAC Field Scanner Season 4 Range 26 Column 15,26,15
309458,MAC Field Scanner Season 4 Range 18 Column 7,33.075159,-111.974950,2017 Jun 29,surface_temperature,35.698114,PI570047,2017-06-29,MAC Field Scanner Season 4 Range 18 Column 7,18,7
297863,MAC Field Scanner Season 4 Range 37 Column 11,33.075842,-111.974884,2017 Jun 4,surface_temperature,36.667383,PI152694,2017-06-04,MAC Field Scanner Season 4 Range 37 Column 11,37,11
117426,MAC Field Scanner Season 4 Range 53 Column 2,33.076417,-111.975032,2017 Jul 21,surface_temperature,33.070886,Big_Kahuna,2017-07-21,MAC Field Scanner Season 4 Range 53 Column 2,53,2
248800,MAC Field Scanner Season 4 Range 48 Column 2,33.076237,-111.975032,2017 Jul 18,leaf_angle_chi,2.132514,PI620157,2017-07-18,MAC Field Scanner Season 4 Range 48 Column 2,48,2


#### Select for traits to be used for `gdd_to_` calculations

In [20]:
traits_to_keep = ['seedling_emergence_rate', 'flag_leaf_emergence_time', 'flowering_time']
df_5 = df_4.loc[df_4.trait.isin(traits_to_keep)]
df_5.shape

(412, 11)

### I. Add Growing Degree Days

In [38]:
season_4_weather_data = pd.read_csv('data/processed/weather_station_season_4_2019-12-20T190949.csv')
# season_4_weather_data.head()

,date,day_of_year,air_temp_max,air_temp_min,air_temp_mean,rh_max,rh_min,rh_mean,precip_total,wind_speed_mean,wind_vector_magnitude,wind_vector_direction,wind_direction_std,max_wind_speed,gdd,cum_precip
0,2017-04-20,110,33.3,14.1,23.5,45.0,5.1,18.2,0.0,1.9,0.8,233,60,8.2,14.0,0.0
1,2017-04-21,111,34.4,11.1,24.0,46.5,5.5,17.2,0.0,2.2,1.3,274,52,8.5,26.0,0.0
2,2017-04-22,112,35.5,14.5,25.0,32.5,6.4,15.6,0.0,1.6,0.5,178,66,5.2,41.0,0.0
3,2017-04-23,113,37.0,12.6,26.5,48.0,6.8,17.5,0.0,2.1,1.1,221,57,8.2,56.0,0.0
4,2017-04-24,114,33.7,14.9,25.7,33.1,8.3,17.5,0.0,3.0,2.0,214,46,9.9,71.0,0.0


In [47]:
season_4_gdd = season_4_weather_data[['date', 'gdd']]
print(season_4_gdd.shape)
season_4_gdd.head()

(150, 2)


,date,gdd
0,2017-04-20,14.0
1,2017-04-21,26.0
2,2017-04-22,41.0
3,2017-04-23,56.0
4,2017-04-24,71.0


In [48]:
season_4_gdd_1 = season_4_gdd.copy()
season_4_gdd_1.date = pd.to_datetime(season_4_gdd_1.date)
# season_4_gdd_1.head()

,date,gdd
0,2017-04-20,14.0
1,2017-04-21,26.0
2,2017-04-22,41.0
3,2017-04-23,56.0
4,2017-04-24,71.0


In [54]:
merged_df_0 = df_5.merge(season_4_gdd_1, how='left', left_on='date_1', right_on='date')
# merged_df_0.head()

,sitename,lat,lon,date_x,trait,mean,cultivar,date_1,sitename_1,range,column,date_y,gdd
0,MAC Field Scanner Season 4 Range 51 Column 5 W,33.076345,-111.974987,2017 May 8 (America/Phoenix),seedling_emergence_rate,0.770,PI641825,2017-05-08,MAC Field Scanner Season 4 Range 51 Column 5,51,5,2017-05-08,257.0
1,MAC Field Scanner Season 4 Range 3 Column 3 W,33.074619,-111.975019,2017 Apr 25 (America/Phoenix),seedling_emergence_rate,0.538,PI22913,2017-04-25,MAC Field Scanner Season 4 Range 3 Column 3,3,3,2017-04-25,84.0
2,MAC Field Scanner Season 4 Range 3 Column 3 W,33.074619,-111.975019,2017 Apr 30 (America/Phoenix),seedling_emergence_rate,0.897,PI22913,2017-04-30,MAC Field Scanner Season 4 Range 3 Column 3,3,3,2017-04-30,142.0
3,MAC Field Scanner Season 4 Range 3 Column 3 W,33.074619,-111.975019,2017 May 4 (America/Phoenix),seedling_emergence_rate,0.910,PI22913,2017-05-04,MAC Field Scanner Season 4 Range 3 Column 3,3,3,2017-05-04,204.0
4,MAC Field Scanner Season 4 Range 3 Column 3 E,33.074619,-111.975011,2017 Apr 25 (America/Phoenix),seedling_emergence_rate,0.362,PI22913,2017-04-25,MAC Field Scanner Season 4 Range 3 Column 3,3,3,2017-04-25,84.0


### II. Drop, Rename, and Reorder Columns

In [58]:
merged_df_0.columns

Index(['sitename', 'lat', 'lon', 'date_x', 'trait', 'mean', 'cultivar',
       'date_1', 'sitename_1', 'range', 'column', 'date_y', 'gdd'],
      dtype='object')

In [59]:
cols_to_drop = ['sitename', 'date_x', 'date_y']
df_6 = merged_df_0.drop(labels=cols_to_drop, axis=1)
# df_6.head()

,lat,lon,trait,mean,cultivar,date_1,sitename_1,range,column,gdd
0,33.076345,-111.974987,seedling_emergence_rate,0.770,PI641825,2017-05-08,MAC Field Scanner Season 4 Range 51 Column 5,51,5,257.0
1,33.074619,-111.975019,seedling_emergence_rate,0.538,PI22913,2017-04-25,MAC Field Scanner Season 4 Range 3 Column 3,3,3,84.0
2,33.074619,-111.975019,seedling_emergence_rate,0.897,PI22913,2017-04-30,MAC Field Scanner Season 4 Range 3 Column 3,3,3,142.0
3,33.074619,-111.975019,seedling_emergence_rate,0.910,PI22913,2017-05-04,MAC Field Scanner Season 4 Range 3 Column 3,3,3,204.0
4,33.074619,-111.975011,seedling_emergence_rate,0.362,PI22913,2017-04-25,MAC Field Scanner Season 4 Range 3 Column 3,3,3,84.0


In [61]:
df_7 = df_6.rename({'date_1': 'date', 'sitename_1': 'sitename', 'mean': 'value'}, axis=1)
# df_7.head()

,lat,lon,trait,value,cultivar,date,sitename,range,column,gdd
0,33.076345,-111.974987,seedling_emergence_rate,0.770,PI641825,2017-05-08,MAC Field Scanner Season 4 Range 51 Column 5,51,5,257.0
1,33.074619,-111.975019,seedling_emergence_rate,0.538,PI22913,2017-04-25,MAC Field Scanner Season 4 Range 3 Column 3,3,3,84.0
2,33.074619,-111.975019,seedling_emergence_rate,0.897,PI22913,2017-04-30,MAC Field Scanner Season 4 Range 3 Column 3,3,3,142.0
3,33.074619,-111.975019,seedling_emergence_rate,0.910,PI22913,2017-05-04,MAC Field Scanner Season 4 Range 3 Column 3,3,3,204.0
4,33.074619,-111.975011,seedling_emergence_rate,0.362,PI22913,2017-04-25,MAC Field Scanner Season 4 Range 3 Column 3,3,3,84.0


In [62]:
new_col_order = ['sitename', 'range', 'column', 'lat', 'lon', 'cultivar', 'trait', 'value', 'date', 'gdd']
df_8 = pd.DataFrame(data=df_7, columns=new_col_order, index=df_7.index)
print(df_8.shape)
# df_8.head()

(412, 10)


,sitename,range,column,lat,lon,cultivar,trait,value,date,gdd
0,MAC Field Scanner Season 4 Range 51 Column 5,51,5,33.076345,-111.974987,PI641825,seedling_emergence_rate,0.770,2017-05-08,257.0
1,MAC Field Scanner Season 4 Range 3 Column 3,3,3,33.074619,-111.975019,PI22913,seedling_emergence_rate,0.538,2017-04-25,84.0
2,MAC Field Scanner Season 4 Range 3 Column 3,3,3,33.074619,-111.975019,PI22913,seedling_emergence_rate,0.897,2017-04-30,142.0
3,MAC Field Scanner Season 4 Range 3 Column 3,3,3,33.074619,-111.975019,PI22913,seedling_emergence_rate,0.910,2017-05-04,204.0
4,MAC Field Scanner Season 4 Range 3 Column 3,3,3,33.074619,-111.975011,PI22913,seedling_emergence_rate,0.362,2017-04-25,84.0


In [ ]:
flag_leaf_df_0 = df_8.loc[]

In [29]:
day_of_planting = datetime.date(2017, 4, 20)
day_of_planting

datetime.date(2017, 4, 20)

### Final Steps: Save as `.csv` file
* Check output filename
* Change name of df

In [ ]:
need_to_create_csv = False

if need_to_create_csv:

    timestamp = datetime.datetime.now().replace(microsecond=0).isoformat()
    output_filename = f'gdd_values_season_4_{timestamp}.csv'.replace(':', '')
    end_of_season_3.to_csv(f'data/processed/{output_filename}')